# Breast Cancer Classification

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [2]:
df=pd.read_csv("Potential datasets for recruitment.csv")
df.head()

Sample code number  Clump Thickness  Uniformity of Cell Size  \
0             1000025                5                        1   
1             1002945                5                        4   
2             1015425                3                        1   
3             1016277                6                        8   
4             1017023                4                        1   

   Uniformity of Cell Shape  Marginal Adhesion  Single Epithelial Cell Size  \
0                         1                  1                            2   
1                         4                  5                            7   
2                         1                  1                            2   
3                         8                  1                            3   
4                         1                  3                            2   

  Bare Nuclei  Bland Chromatin  Normal Nucleoli  Mitoses  \
0           1                3                1        1   
1          10                3                2        1   
2           2                3                1        1   
3           4                3                7        1   
4           1                3                1        1   

   Class: (2 for benign,  4 for malignant)   4 for malignant)  Unnamed: 12  \
0                                        2                NaN          NaN   
1                                        2                NaN          NaN   
2                                        2                NaN          NaN   
3                                        2                NaN          NaN   
4                                        2                NaN          NaN   

   Unnamed: 13  Unnamed: 14  Unnamed: 15  
0          NaN          NaN          NaN  
1          NaN          NaN          NaN  
2          NaN          NaN          NaN  
3          NaN          NaN          NaN  
4          NaN          NaN          NaN

In [3]:
#prominent nucleoli, many mitoses
df.columns

Index(['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size',
       'Uniformity of Cell Shape', 'Marginal Adhesion',
       'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
       'Normal Nucleoli', 'Mitoses', 'Class: (2 for benign,  4 for malignant)',
       ' 4 for malignant)', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15'],
      dtype='object')

In [4]:
df.shape

(699, 16)

In [5]:
dataframe=df.iloc[:, 0:11]
dataframe.isnull().sum()
for i in range(0, dataframe.shape[0]):
    if(dataframe['Bare Nuclei'][i]=='?'):
        dataframe['Bare Nuclei'][i]=0    
dataframe['Bare Nuclei'].astype(str).astype(int)
print()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [6]:
X_v=dataframe.iloc[:, 1:10]
Y_v=dataframe.iloc[:, 10:]

x_tr, x_te, y_tr, y_te=train_test_split(X_v, Y_v, test_size=0.25, random_state=0)

clf=LogisticRegression()
clf.fit(x_tr, y_tr)
y_pr=clf.predict(x_te)
print("ini score",  accuracy_score(y_pr, y_te))

print("comfusion matrix ini: \n", confusion_matrix(y_te, y_pr, labels=[2, 4]))

ini score 0.9657142857142857
comfusion matrix ini: 
 [[110   2]
 [  4  59]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
del(dataframe['Normal Nucleoli'])
del(dataframe['Mitoses'])
dataframe.shape

X_val=dataframe.iloc[:, 1:8]
Y_val=dataframe.iloc[:, 8:]
sc=StandardScaler()
X_trans=sc.fit_transform(X_val)

# To apply PCA on the reduced Dataset

In [8]:
import numpy as np
all_data=X_trans
all_data_t=X_trans.T#transpose
covar=np.cov(all_data_t)
eigen_val, eigen_vectors=np.linalg.eig(covar)

In [9]:
eigen_val_vector_pair=[]
for i in range(len(eigen_val)):
    eig_vec=eigen_vectors[:, i]
    eigen_val_vector_pair.append((eigen_val[i], eig_vec))
eigen_val_vector_pair #first component is eigen vector and second being eigen value

[(5.003285356986542,
  array([-0.33400351, -0.41535668, -0.41194042, -0.36404622, -0.36321666,
         -0.37053769, -0.38009394])),
 (0.5385878144303353,
  array([ 8.59286969e-01, -2.86218208e-02,  3.24683236e-02, -4.52002510e-01,
         -1.71194192e-01, -8.37673570e-04, -1.61672760e-01])),
 (0.08917323462749692,
  array([ 0.00182663,  0.7401799 , -0.66147481, -0.04629468, -0.06842161,
          0.07059775, -0.05265648])),
 (0.46723352814815833,
  array([-0.08924475,  0.24831649,  0.19116968, -0.36066939,  0.67135543,
         -0.52984634, -0.17969633])),
 (0.2630851481786608,
  array([-0.23503057,  0.41662107,  0.56901598, -0.09555172, -0.42895855,
          0.1271525 , -0.48796004])),
 (0.31201041503297583,
  array([ 0.27773931,  0.08430746,  0.0141329 ,  0.65981205, -0.19632774,
         -0.64262452, -0.16938301])),
 (0.33665315589095895,
  array([-0.09859569,  0.19092248,  0.17657067, -0.29403471, -0.40101939,
         -0.38450735,  0.72631359]))]

#### Based on the above eigen vectors we can include 3 n_components

In [10]:
pca1=PCA(n_components=5)
pca2=PCA(n_components=3)
X_pca1=pca1.fit_transform(X_trans)
X_pca2=pca2.fit_transform(X_trans)

x_train1, x_test1, y_train1, y_test1=train_test_split(X_pca1, Y_val, test_size=0.25)
x_train2, x_test2, y_train2, y_test2=train_test_split(X_pca2, Y_val, test_size=0.25)
dataframe["Class: (2 for benign,  4 for malignant)"].unique()

array([2, 4], dtype=int64)

## Applying the Logistic Regression on both the train datas i.e X_pca1, X_pca2

In [11]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(x_train1, y_train1)
y_pred1=clf.predict(x_test1)

clf.fit(x_train2, y_train2)
y_pred2=clf.predict(x_test2)
print(accuracy_score(y_pred2, y_test2))
print(confusion_matrix(y_test1, y_pred1, labels=[2, 4]))


0.9714285714285714
[[120   2]
 [  2  51]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
print(accuracy_score(y_pred1, y_test1))
print(confusion_matrix(y_test2, y_pred2, labels=[2, 4]))
#Improve your classification model to reduce patients who are being predicted as having benign tumor but actually have malignant tumor.

0.9771428571428571
[[122   3]
 [  2  48]]


# Alternate solution: KNN

In [13]:
dataframe=df.iloc[:, 0:11]
dataframe.isnull().sum()
for i in range(0, dataframe.shape[0]):
    if(dataframe['Bare Nuclei'][i]=='?'):
        dataframe['Bare Nuclei'][i]=0    
dataframe['Bare Nuclei'].astype(str).astype(int)
print()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [14]:
X_val=dataframe.iloc[:, 1:10]
Y_val=dataframe.iloc[:, 10:]
sc=StandardScaler()
X_trans=sc.fit_transform(X_val)

x_axis=[]
y_axis=[]

x_train, x_test, y_train, y_test=train_test_split(X_val, Y_val, test_size=0.25, random_state=0)
for i in range(1, 9):
    clf=KNeighborsClassifier(n_neighbors=i)
    score=cross_val_score(clf, x_train, y_train)
    x_axis.append(i)
    y_axis.append(score.mean())
    
import matplotlib.pyplot as plt
plt.plot(x_axis, y_axis)
plt.show()

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y

<Figure size 640x480 with 1 Axes>

In [15]:
#### Hence we choose k=3 infering from above graph
clf1=KNeighborsClassifier(n_neighbors=3)
clf1.fit(x_train, y_train)
y_pred_knn=clf1.predict(x_test)
accuracy_score(y_pred_knn, y_test)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9771428571428571

In [16]:
print(confusion_matrix(y_test, y_pred_knn, labels=[2, 4]))

[[110   2]
 [  2  61]]
